In [3]:
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Загрузка датасета
data = pd.read_csv('diabetes.csv')

# Разделение данных на признаки и целевую переменную
X = data.drop("Outcome", axis=1)
y = data["Outcome"]

# Балансировка классов (если данных о диабете меньше)
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Масштабирование данных
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Настройка XGBoost с подбором гиперпараметров
params = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0]
}

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
grid_search = GridSearchCV(model, params, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Лучшая модель
best_model = grid_search.best_estimator_

# Проверка точности
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy:.2f}")

# Сохранение модели
joblib.dump((best_model, scaler), "diabetes_model.pkl")

def predict_diabetes(input_data):
    model, scaler = joblib.load("diabetes_model.pkl")
    input_array = np.array(input_data).reshape(1, -1)
    input_array = scaler.transform(input_array)  # Применяем масштабирование
    prediction = model.predict(input_array)[0]
    probability = model.predict_proba(input_array)[0][1] * 100
    
    if prediction == 1:
        result = f"Диабет обнаружен (вероятность: {probability:.2f}%). Вам следует проконсультироваться с врачом."
    elif probability > 50:
        result = f"Есть склонность к диабету (вероятность: {probability:.2f}%). Рекомендуется следить за питанием и образом жизни."
    else:
        result = f"Диабет не обнаружен (вероятность: {probability:.2f}%)."
    
    return result

# Примеры пользовательского ввода
user_input_healthy = [2, 120, 70, 20, 85, 25.5, 0.3, 30]  # Данные без диабета
user_input_diabetic = [8, 180, 90, 40, 200, 35.0, 0.8, 50]  # Данные с диабетом

print("Результат для здорового пользователя:")
print(predict_diabetes(user_input_healthy))

print("\nРезультат для пользователя с диабетом:")
print(predict_diabetes(user_input_diabetic))

Fitting 3 folds for each of 24 candidates, totalling 72 fits
Точность модели: 0.78
Результат для здорового пользователя:
Диабет не обнаружен (вероятность: 4.70%).

Результат для пользователя с диабетом:
Диабет обнаружен (вероятность: 98.17%). Вам следует проконсультироваться с врачом.


C:\Users\Dos\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [14:57:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
d:\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
d:\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [6]:
def test_predict_diabetes():
    test_cases = [
        {
            "input": [2, 120, 70, 20, 85, 25.5, 0.3, 30],
            "description": "Корректные данные (здоровый пользователь)"
        },
        {
            "input": [8, 180, 90, 40, 200, 35.0, 0.8, 50],
            "description": "Корректные данные (пользователь с диабетом)"
        },
        {
            "input": [5, 100, 70],  # Недостаточно признаков
            "description": "Недостаточное количество признаков"
        },
        {
            "input": "not a list",  # Неверный формат
            "description": "Неверный тип данных"
        }
    ]

    for i, case in enumerate(test_cases, 1):
        print(f"\nТест {i}: {case['description']}")
        try:
            result = predict_diabetes(case["input"])
            print("Результат:", result)
        except Exception as e:
            print("Ошибка:", str(e))


In [7]:
test_predict_diabetes()


Тест 1: Корректные данные (здоровый пользователь)
Результат: Диабет не обнаружен (вероятность: 4.70%).

Тест 2: Корректные данные (пользователь с диабетом)
Результат: Диабет обнаружен (вероятность: 98.17%). Вам следует проконсультироваться с врачом.

Тест 3: Недостаточное количество признаков
Ошибка: X has 3 features, but StandardScaler is expecting 8 features as input.

Тест 4: Неверный тип данных
Ошибка: could not convert string to float: 'not a list'


d:\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
d:\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
d:\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
d:\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
